In [1]:
import pandas
from datetime import datetime, timedelta
from dateutil.relativedelta import *
import time
from multiprocessing import Process
from threading import Thread
import numpy as np

# Функции расчета фич

In [2]:
# Для увеличения даты в зависимости от вида смещения
def uvelichenieDate(startDate, period):
    if period == 'day':
        return startDate + relativedelta(days = 1)
    if period == 'week':
        return startDate + relativedelta(days = 7)
    if period == 'months':
        return startDate + relativedelta(months = 1)
    if period == 'quarted':
        return startDate + relativedelta(months = 3)
    if period == 'year':
        return startDate + relativedelta(years = 1)

In [3]:
# Для вычисления начальной даты
def nachalnayaData(FinishDate, StartDate, period):
    # Если период all, то нет смысла вычислять начальную дату, т.к. статистику будем брать за весь период
    if period == 'all':
        return StartDate
    
    timeDate = FinishDate
    # цикл смещения, преднозначен для вычисления начальной даты относительно "Последняя дата" - период
    while timeDate >= StartDate:
        if period == 'week':
            timeDate = timeDate + relativedelta(days = -7)
        elif period == 'months':
            timeDate = timeDate + relativedelta(months = -1)
        elif period == 'quarted':
            timeDate = timeDate + relativedelta(months = -3)
        elif period == 'year':
            timeDate = timeDate + relativedelta(years = -1)
    return timeDate

In [4]:
# period - с каким шагом периода будем группировать данные
# grupBaza - столбец основание группировки
# grupPriznak - столбец с дополнительным разрезом группировки
# metodGrup - сумма или количество
# newColunmName - имя нового столбца в который добавим данные
# Глобально, 
    # 1) разбиваем DF по временным отрезкам.
    # 2) Вычисляем признаки (сумма или т.п.).
    # 3) Записываем в DF

def makePriznak(df, period, grupBaza, grupPriznak, metodGrup, newColunmName):
    # добавим новый столбец в который будем записывать значения фич
    df[newColunmName] = None 
    # Правая граница цикла
    # df['Date'][len(df)-1] последняя запись в DF
    endDateOfDF = uvelichenieDate(df['ZakazData'][len(df)-1], "day")
    # Получим первую дату указанную в ДФ (она же левая граница цикла)
    firstDate = nachalnayaData(endDateOfDF, df['ZakazData'][0], period)
    
    # Расчитаем вторую дату
    # если не равно all, то берем дату за period, 
    # в противном случае данные за весь период
    if period != 'all':
        secondDate = uvelichenieDate(firstDate, period)
    else:
        secondDate = endDateOfDF
    
    # цикл для прохода DF по условию дат
    while firstDate <= endDateOfDF:
        # Отберем данные по условию дат
        filtredDF = df.loc[(df['ZakazData'] >= firstDate) & 
                           (df['ZakazData'] < secondDate)]
        
        #Сделаем dict уникальных значений по виду столбца
        timeDict = dict.fromkeys(filtredDF[grupBaza])
        
        # идем по уникальным значениям словаря
        for elem in timeDict:
            #фильтруем по уникальным значениям
            filtredDF2 = filtredDF.loc[(df[grupBaza] == elem)]
            
            #Обрабатываем значения по metodGrup (Сумма или кол-во и т.п.)
            if metodGrup == 'sum':
                rezult = filtredDF2[grupPriznak].sum()
            elif metodGrup == 'max':
                rezult = filtredDF2[grupPriznak].max()
            elif metodGrup == 'median':
                rezult = filtredDF2[grupPriznak].median()
            elif metodGrup == 'pct_change':
                rezult = filtredDF2[grupPriznak].pct_change()
            elif metodGrup == 'nunique':
                rezult = filtredDF2[grupPriznak].nunique()
            
            # Записываем данные в DF
            df.at[filtredDF2.index.values, newColunmName] = rezult
        
        # Если period = all, то нет смысла дальше считать, 
        # т.к. при all считаются все данные на первой итерации
        if period == 'all':
            break

        # сдвинем границы на значение period
        firstDate = uvelichenieDate(firstDate, period) 
        secondDate = uvelichenieDate(firstDate, period)
        
    print(f'Выполнен расчет фич для: {newColunmName}')

# Загрузим данные

In [5]:
file = pandas.read_excel('Продажи с касаниями клиентов.xlsx', index_col=0)

In [6]:
timeFile = file.copy(deep=True)

In [7]:
timeFile[:1]

,AktVipRabot,Data,SummaAktaVipRabot,Quantity,KodStrokiAkt,ZakazKlienta,NomenklaturaKod,Nomenklatura,ZakazData,PartnerKod,...,ProdanoShtuk,KodStrokiRealiz,LinkOrder,SummaZakazano,RastoynieOtNSK,otMoscow,ПродажаСостоялась,ItogoProcentSkidki,cals,emails
0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2014-06-10 12:00:00,000001587,...,NaN,NaN,Заказ клиента ЛБ00-000001 от 10.06.2014 12:00:00,3690.0,1,3400,Нет,NaN,0,0


In [8]:
timeFile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146016 entries, 0 to 146015
Data columns (total 44 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   AktVipRabot                  2375 non-null    object        
 1   Data                         2375 non-null    datetime64[ns]
 2   SummaAktaVipRabot            2375 non-null    float64       
 3   Quantity                     2375 non-null    float64       
 4   KodStrokiAkt                 2375 non-null    float64       
 5   ZakazKlienta                 2375 non-null    object        
 6   NomenklaturaKod              2375 non-null    object        
 7   Nomenklatura                 2375 non-null    object        
 8   ZakazData                    146016 non-null  datetime64[ns]
 9   PartnerKod                   146016 non-null  object        
 10  OsnovnoyManager              146016 non-null  object        
 11  BiznesRegion              

# Выполним расчет фич

## Заказ - как основной признак

In [9]:
# Количество уникальной номенклатуры по Заказу
makePriznak(timeFile, "week", "LinkOrder", 'Nomenklatura', "nunique", 'УникНоменкЗаказНеделя')
makePriznak(timeFile, "months", "LinkOrder", 'Nomenklatura', "nunique", 'УникНоменкЗаказМесяц')
makePriznak(timeFile, "quarted", "LinkOrder", 'Nomenklatura', "nunique", 'УникНоменкЗаказКвартал')
makePriznak(timeFile, "year", "LinkOrder", 'Nomenklatura', "nunique", 'УникНоменкЗаказГод')
makePriznak(timeFile, "all", "LinkOrder", 'Nomenklatura', "nunique", 'УникНоменкЗаказВсего')

# Сумма заказа
# Таким образом просто перепишем столбец SummaZakaza
makePriznak(timeFile, "all", "LinkOrder", 'SummaZakazano', "sum", 'SummaZakaza')

Выполнен расчет фич для: УникНоменкЗаказНеделя
Выполнен расчет фич для: УникНоменкЗаказМесяц
Выполнен расчет фич для: УникНоменкЗаказКвартал
Выполнен расчет фич для: УникНоменкЗаказГод
Выполнен расчет фич для: УникНоменкЗаказВсего
Выполнен расчет фич для: SummaZakaza


## Партнер - как основной признак

In [10]:
# Количество запроссов (Заказов) в заданный период (частота обращений)
makePriznak(timeFile, "week", "PartnerKod", 'LinkOrder', "nunique", 'Кол_воЗапроссовВНеделюУПартнера')
makePriznak(timeFile, "months", "PartnerKod", 'LinkOrder', "nunique", 'Кол_воЗапроссовВМесяцУПартнера')
makePriznak(timeFile, "quarted", "PartnerKod", 'LinkOrder', "nunique", 'Кол_воЗапроссовВКварталУПартнера')
makePriznak(timeFile, "year", "PartnerKod", 'LinkOrder', "nunique", 'Кол_воЗапроссовВГодУПартнера')
makePriznak(timeFile, "all", "PartnerKod", 'LinkOrder', "nunique", 'Кол_воЗапроссовВсегоУПартнера')

# Количество реализаций в заданный период (частота продаж)
makePriznak(timeFile, "week", "PartnerKod", 'RealizSsilka', "nunique", 'Кол_воРеализацПартнерВНеделю')
makePriznak(timeFile, "months", "PartnerKod", 'RealizSsilka', "nunique", 'Кол_воРеализацПартнерВМесяц')
makePriznak(timeFile, "quarted", "PartnerKod", 'RealizSsilka', "nunique", 'Кол_воРеализацПартнерВКвартал')
makePriznak(timeFile, "year", "PartnerKod", 'RealizSsilka', "nunique", 'Кол_воРеализацПартнерВГод')
makePriznak(timeFile, "all", "PartnerKod", 'RealizSsilka', "nunique", 'Кол_воРеализацПартнерВсего')

# Количество реализаций в заданный период по номенклатуре (частота продаж)
makePriznak(timeFile, "week", "PartnerKod", 'NomenklaturaKodRealAndZakaz', "nunique", 'АссорНоменклатПартнерНеделя')
makePriznak(timeFile, "months", "PartnerKod", 'NomenklaturaKodRealAndZakaz', "nunique", 'АссорНоменклатПартнерМесяц')
makePriznak(timeFile, "quarted", "PartnerKod", 'NomenklaturaKodRealAndZakaz', "nunique", 'АссорНоменклатПартнерКвартал')
makePriznak(timeFile, "year", "PartnerKod", 'NomenklaturaKodRealAndZakaz', "nunique", 'АссорНоменклатПартнерГод')
makePriznak(timeFile, "all", "PartnerKod", 'NomenklaturaKodRealAndZakaz', "nunique", 'АссорНоменклатПартнерВсего')

# Количество уникальных Видов номенклатуры по Партнеру
makePriznak(timeFile, "week", "PartnerKod", 'VidNomenklatury', "nunique", 'УникВидНоменкПартнерНеделя')
makePriznak(timeFile, "months", "PartnerKod", 'VidNomenklatury', "nunique", 'УникВидНоменкПартнерМесяц')
makePriznak(timeFile, "quarted", "PartnerKod", 'VidNomenklatury', "nunique", 'УникВидНоменкПартнерКвартал')
makePriznak(timeFile, "year", "PartnerKod", 'VidNomenklatury', "nunique", 'УникВидНоменкПартнерГод')
makePriznak(timeFile, "all", "PartnerKod", 'VidNomenklatury', "nunique", 'УникВидНоменкПартнерВсего')

# Оборот по Партнеру ЗаказаноВсего
makePriznak(timeFile, "week", "PartnerKod", 'SummaZakaza', "sum", 'ЗаказСуммаПартнерНеделя')
makePriznak(timeFile, "months", "PartnerKod", 'SummaZakaza', "sum", 'ЗаказСуммаПартнерНеделяМесяц')
makePriznak(timeFile, "quarted", "PartnerKod", 'SummaZakaza', "sum", 'ЗаказСуммаПартнерНеделяКвартал')
makePriznak(timeFile, "year", "PartnerKod", 'SummaZakaza', "sum", 'ЗаказСуммаПартнерНеделяГод')
makePriznak(timeFile, "all", "PartnerKod", 'SummaZakaza', "sum", 'ЗаказСуммаПартнерНеделяВсего')

# Звонков по Партнеру
makePriznak(timeFile, "week", "PartnerKod", 'cals', "sum", 'ЗвонковПартнерНеделя')
makePriznak(timeFile, "months", "PartnerKod", 'cals', "sum", 'ЗвонковПартнерНеделяМесяц')
makePriznak(timeFile, "quarted", "PartnerKod", 'cals', "sum", 'ЗвонковПартнерНеделяКвартал')
makePriznak(timeFile, "year", "PartnerKod", 'cals', "sum", 'ЗвонковПартнерНеделяГод')
makePriznak(timeFile, "all", "PartnerKod", 'cals', "sum", 'ЗвонковПартнерНеделяВсего')

# email по Партнеру ЗаказаноВсего
makePriznak(timeFile, "week", "PartnerKod", 'emails', "sum", 'ЭлПочтыПартнерНеделя')
makePriznak(timeFile, "months", "PartnerKod", 'emails', "sum", 'ЭлПочтыПартнерНеделяМесяц')
makePriznak(timeFile, "quarted", "PartnerKod", 'emails', "sum", 'ЭлПочтыПартнерНеделяКвартал')
makePriznak(timeFile, "year", "PartnerKod", 'emails', "sum", 'ЭлПочтыПартнерНеделяГод')
makePriznak(timeFile, "all", "PartnerKod", 'emails', "sum", 'ЭлПочтыПартнерНеделяВсего')

Выполнен расчет фич для: Кол_воЗапроссовВНеделюУПартнера
Выполнен расчет фич для: Кол_воЗапроссовВМесяцУПартнера
Выполнен расчет фич для: Кол_воЗапроссовВКварталУПартнера
Выполнен расчет фич для: Кол_воЗапроссовВГодУПартнера
Выполнен расчет фич для: Кол_воЗапроссовВсегоУПартнера
Выполнен расчет фич для: Кол_воРеализацПартнерВНеделю
Выполнен расчет фич для: Кол_воРеализацПартнерВМесяц
Выполнен расчет фич для: Кол_воРеализацПартнерВКвартал
Выполнен расчет фич для: Кол_воРеализацПартнерВГод
Выполнен расчет фич для: Кол_воРеализацПартнерВсего
Выполнен расчет фич для: АссорНоменклатПартнерНеделя
Выполнен расчет фич для: АссорНоменклатПартнерМесяц
Выполнен расчет фич для: АссорНоменклатПартнерКвартал
Выполнен расчет фич для: АссорНоменклатПартнерГод
Выполнен расчет фич для: АссорНоменклатПартнерВсего
Выполнен расчет фич для: УникВидНоменкПартнерНеделя
Выполнен расчет фич для: УникВидНоменкПартнерМесяц
Выполнен расчет фич для: УникВидНоменкПартнерКвартал
Выполнен расчет фич для: УникВидНоменк

## БизнесРегион - как основной признак

In [11]:
# Ассортимент Номенклатуры по БизнесРегиону
makePriznak(timeFile, "week", "BiznesRegion", 'NomenklaturaKodRealAndZakaz', "nunique", 'АссортНоменклПоБизнесРегионуНеделя')
makePriznak(timeFile, "months", "BiznesRegion", 'NomenklaturaKodRealAndZakaz', "nunique", 'АссортНоменклПоБизнесРегионуМесяц')
makePriznak(timeFile, "quarted", "BiznesRegion", 'NomenklaturaKodRealAndZakaz', "nunique", 'АссортНоменклПоБизнесРегионуКвартал')
makePriznak(timeFile, "year", "BiznesRegion", 'NomenklaturaKodRealAndZakaz', "nunique", 'АссортНоменклПоБизнесРегионуГод')
makePriznak(timeFile, "all", "BiznesRegion", 'NomenklaturaKodRealAndZakaz', "nunique", 'АссортНоменклПоБизнесРегионуВсего')

# Количество Заказов по БизнесРегиону
makePriznak(timeFile, "week", "BiznesRegion", 'LinkOrder', "nunique", 'УникЗаказБизнесРегионНеделя')
makePriznak(timeFile, "months", "BiznesRegion", 'LinkOrder', "nunique", 'УникЗаказБизнесРегионМесяц')
makePriznak(timeFile, "quarted", "BiznesRegion", 'LinkOrder', "nunique", 'УникЗаказБизнесРегионКвартал')
makePriznak(timeFile, "year", "BiznesRegion", 'LinkOrder', "nunique", 'УникЗаказБизнесРегионГод')
makePriznak(timeFile, "all", "BiznesRegion", 'LinkOrder', "nunique", 'УникЗаказБизнесРегионВсего')

# Количество Реализций по БизнесРегиону
makePriznak(timeFile, "week", "BiznesRegion", 'RealizSsilka', "nunique", 'УникРеализБизнесРегионНеделя')
makePriznak(timeFile, "months", "BiznesRegion", 'RealizSsilka', "nunique", 'УникРеализБизнесРегионМесяц')
makePriznak(timeFile, "quarted", "BiznesRegion", 'RealizSsilka', "nunique", 'УникРеализБизнесРегионКвартал')
makePriznak(timeFile, "year", "BiznesRegion", 'RealizSsilka', "nunique", 'УникРеализБизнесРегионГод')
makePriznak(timeFile, "all", "BiznesRegion", 'RealizSsilka', "nunique", 'УникРеализБизнесРегионВсего')

# Количество Партнеров в БизнесРегиону
makePriznak(timeFile, "week", "BiznesRegion", 'PartnerKod', "nunique", 'Кол_воПартнеровВРегионеНеделя')
makePriznak(timeFile, "months", "BiznesRegion", 'PartnerKod', "nunique", 'Кол_воПартнеровВРегионеМесяц')
makePriznak(timeFile, "quarted", "BiznesRegion", 'PartnerKod', "nunique", 'Кол_воПартнеровВРегионеКвартал')
makePriznak(timeFile, "year", "BiznesRegion", 'PartnerKod', "nunique", 'Кол_воПартнеровВРегионеГод')
makePriznak(timeFile, "all", "BiznesRegion", 'PartnerKod', "nunique", 'Кол_воПартнеровВРегионеВсего')

# Количество сотрудников (менеджеров) работающих с БизнесРегионом
makePriznak(timeFile, "week", "BiznesRegion", 'OsnovnoyManager', "nunique", 'Кол_воОснМенеджВБизнесРегионНеделя')
makePriznak(timeFile, "months", "BiznesRegion", 'OsnovnoyManager', "nunique", 'Кол_воОснМенеджВБизнесРегионМесяц')
makePriznak(timeFile, "quarted", "BiznesRegion", 'OsnovnoyManager', "nunique", 'Кол_воОснМенеджВБизнесРегионКвартал')
makePriznak(timeFile, "year", "BiznesRegion", 'OsnovnoyManager', "nunique", 'Кол_воОснМенеджВБизнесРегионГод')
makePriznak(timeFile, "all", "BiznesRegion", 'OsnovnoyManager', "nunique", 'Кол_воОснМенеджВБизнесРегионВсего')

# Сумма Заказа по Бизнес Региону
makePriznak(timeFile, "week", "BiznesRegion", 'SummaZakaza', "sum", 'СуммаЗаказовБизнесРегионНеделя')
makePriznak(timeFile, "months", "BiznesRegion", 'SummaZakaza', "sum", 'СуммаЗаказовБизнесРегионМесяц')
makePriznak(timeFile, "quarted", "BiznesRegion", 'SummaZakaza', "sum", 'СуммаЗаказовБизнесРегионКвартал')
makePriznak(timeFile, "year", "BiznesRegion", 'SummaZakaza', "sum", 'СуммаЗаказовБизнесРегионГод')
makePriznak(timeFile, "all", "BiznesRegion", 'SummaZakaza', "sum", 'СуммаЗаказовБизнесРегионВсего')

# Сумма Реализаций по Бизнес Региону
makePriznak(timeFile, "week", "BiznesRegion", 'SummaRealiz', "sum", 'СуммаРеализацийБизнесРегионНеделя')
makePriznak(timeFile, "months", "BiznesRegion", 'SummaRealiz', "sum", 'СуммаРеализацийБизнесРегионМесяц')
makePriznak(timeFile, "quarted", "BiznesRegion", 'SummaRealiz', "sum", 'СуммаРеализацийБизнесРегионКвартал')
makePriznak(timeFile, "year", "BiznesRegion", 'SummaRealiz', "sum", 'СуммаРеализацийБизнесРегионГод')
makePriznak(timeFile, "all", "BiznesRegion", 'SummaRealiz', "sum", 'СуммаРеализацийБизнесРегионВсего')

# Звонков по Бизне региону
makePriznak(timeFile, "week", "BiznesRegion", 'cals', "sum", 'ЗвонковБизнесРегионНеделя')
makePriznak(timeFile, "months", "BiznesRegion", 'cals', "sum", 'ЗвонковБизнесРегионМесяц')
makePriznak(timeFile, "quarted", "BiznesRegion", 'cals', "sum", 'ЗвонковБизнесРегионКвартал')
makePriznak(timeFile, "year", "BiznesRegion", 'cals', "sum", 'ЗвонковБизнесРегионГод')
makePriznak(timeFile, "all", "BiznesRegion", 'cals', "sum", 'ЗвонковБизнесРегионВсего')

# email по Бизне региону
makePriznak(timeFile, "week", "BiznesRegion", 'emails', "sum", 'ЭлПочтыБизнесРегионНеделя')
makePriznak(timeFile, "months", "BiznesRegion", 'emails', "sum", 'ЭлПочтыБизнесРегионМесяц')
makePriznak(timeFile, "quarted", "BiznesRegion", 'emails', "sum", 'ЭлПочтыБизнесРегионКвартал')
makePriznak(timeFile, "year", "BiznesRegion", 'emails', "sum", 'ЭлПочтыБизнесРегионГод')
makePriznak(timeFile, "all", "BiznesRegion", 'emails', "sum", 'ЭлПочтыБизнесРегионВсего')

Выполнен расчет фич для: АссортНоменклПоБизнесРегионуНеделя
Выполнен расчет фич для: АссортНоменклПоБизнесРегионуМесяц
Выполнен расчет фич для: АссортНоменклПоБизнесРегионуКвартал
Выполнен расчет фич для: АссортНоменклПоБизнесРегионуГод
Выполнен расчет фич для: АссортНоменклПоБизнесРегионуВсего
Выполнен расчет фич для: УникЗаказБизнесРегионНеделя
Выполнен расчет фич для: УникЗаказБизнесРегионМесяц
Выполнен расчет фич для: УникЗаказБизнесРегионКвартал
Выполнен расчет фич для: УникЗаказБизнесРегионГод
Выполнен расчет фич для: УникЗаказБизнесРегионВсего
Выполнен расчет фич для: УникРеализБизнесРегионНеделя
Выполнен расчет фич для: УникРеализБизнесРегионМесяц
Выполнен расчет фич для: УникРеализБизнесРегионКвартал
Выполнен расчет фич для: УникРеализБизнесРегионГод
Выполнен расчет фич для: УникРеализБизнесРегионВсего
Выполнен расчет фич для: Кол_воПартнеровВРегионеНеделя
Выполнен расчет фич для: Кол_воПартнеровВРегионеМесяц
Выполнен расчет фич для: Кол_воПартнеровВРегионеКвартал
Выполнен рас

## Основной Менеджер - как основной признак

In [12]:
# Количество уникальной номенклатуры по Менеджеру
makePriznak(timeFile, "week", "OsnovnoyManager", 'NomenklaturaKodRealAndZakaz', "nunique", 'УникНоменкМенеджНеделя')
makePriznak(timeFile, "months", "OsnovnoyManager", 'NomenklaturaKodRealAndZakaz', "nunique", 'УникНоменкМенеджМесяц')
makePriznak(timeFile, "quarted", "OsnovnoyManager", 'NomenklaturaKodRealAndZakaz', "nunique", 'УникНоменкМенеджКвартал')
makePriznak(timeFile, "year", "OsnovnoyManager", 'NomenklaturaKodRealAndZakaz', "nunique", 'УникНоменкМенеджГод')
makePriznak(timeFile, "all", "OsnovnoyManager", 'NomenklaturaKodRealAndZakaz', "nunique", 'УникНоменкМенеджВсего')

# Количество вида номенклатуры Партнеров по Менеджеру
makePriznak(timeFile, "week", "OsnovnoyManager", 'VidNomenklatury', "nunique", 'УникВидНоменкМенеджерНеделя')
makePriznak(timeFile, "months", "OsnovnoyManager", 'VidNomenklatury', "nunique", 'УникВидНоменкМенеджерМесяц')
makePriznak(timeFile, "quarted", "OsnovnoyManager", 'VidNomenklatury', "nunique", 'УникВидНоменкМенеджерКвартал')
makePriznak(timeFile, "year", "OsnovnoyManager", 'VidNomenklatury', "nunique", 'УникВидНоменкМенеджерГод')
makePriznak(timeFile, "all", "OsnovnoyManager", 'VidNomenklatury', "nunique", 'УникВидНоменкМенеджерВсе')


# Количество уникальных Партнеров по Менеджеру
makePriznak(timeFile, "week", "OsnovnoyManager", 'PartnerKod', "nunique", 'Кол_воПартнеровУМенеджераНеделя')
makePriznak(timeFile, "months", "OsnovnoyManager", 'PartnerKod', "nunique", 'Кол_воПартнеровУМенеджераМесяц')
makePriznak(timeFile, "quarted", "OsnovnoyManager", 'PartnerKod', "nunique", 'Кол_воПартнеровУМенеджераКвартал')
makePriznak(timeFile, "year", "OsnovnoyManager", 'PartnerKod', "nunique", 'Кол_воПартнеровУМенеджераГод')
makePriznak(timeFile, "all", "OsnovnoyManager", 'PartnerKod', "nunique", 'Кол_воПартнеровУМенеджераВсего')

# География поставок Менеджера
makePriznak(timeFile, "week", "OsnovnoyManager", 'BiznesRegion', "nunique", 'МенеджерКол_воРегионовПоставкиНеделя')
makePriznak(timeFile, "months", "OsnovnoyManager", 'BiznesRegion', "nunique", 'МенеджерКол_воРегионовПоставкиМесяц')
makePriznak(timeFile, "quarted", "OsnovnoyManager", 'BiznesRegion', "nunique", 'МенеджерКол_воРегионовПоставкиКвартал')
makePriznak(timeFile, "year", "OsnovnoyManager", 'BiznesRegion', "nunique", 'МенеджерКол_воРегионовПоставкиГод')
makePriznak(timeFile, "all", "OsnovnoyManager", 'BiznesRegion', "nunique", 'МенеджерКол_воРегионовПоставкиВсе')

# Количество Заказов в заданный период по Менеджеру (загрузка менеджера)
makePriznak(timeFile, "week", "OsnovnoyManager", 'LinkOrder', "nunique", 'Кол_воЗаказовВНеделюУМенеджера')
makePriznak(timeFile, "months", "OsnovnoyManager", 'LinkOrder', "nunique", 'Кол_воЗаказовВМесяцУМенеджера')
makePriznak(timeFile, "quarted", "OsnovnoyManager", 'LinkOrder', "nunique", 'Кол_воЗаказовВКварталУМенеджера')
makePriznak(timeFile, "year", "OsnovnoyManager", 'LinkOrder', "nunique", 'Кол_воЗаказовВГодУМенеджера')
makePriznak(timeFile, "all", "OsnovnoyManager", 'LinkOrder', "nunique", 'Кол_воЗаказовВсегоУМенеджера')

# Количество Реализаций в заданный период по Менеджеру (загрузка менеджера)
makePriznak(timeFile, "week", "OsnovnoyManager", 'RealizSsilka', "nunique", 'Кол_воРеализацВНеделюУМенеджера')
makePriznak(timeFile, "months", "OsnovnoyManager", 'RealizSsilka', "nunique", 'Кол_воРеализацВМесяцУМенеджера')
makePriznak(timeFile, "quarted", "OsnovnoyManager", 'RealizSsilka', "nunique", 'Кол_воРеализацВКварталУМенеджера')
makePriznak(timeFile, "year", "OsnovnoyManager", 'RealizSsilka', "nunique", 'Кол_воРеализацВГодУМенеджера')
makePriznak(timeFile, "all", "OsnovnoyManager", 'RealizSsilka', "nunique", 'Кол_воРеализацВсегоУМенеджера')

Выполнен расчет фич для: УникНоменкМенеджНеделя
Выполнен расчет фич для: УникНоменкМенеджМесяц
Выполнен расчет фич для: УникНоменкМенеджКвартал
Выполнен расчет фич для: УникНоменкМенеджГод
Выполнен расчет фич для: УникНоменкМенеджВсего
Выполнен расчет фич для: УникВидНоменкМенеджерНеделя
Выполнен расчет фич для: УникВидНоменкМенеджерМесяц
Выполнен расчет фич для: УникВидНоменкМенеджерКвартал
Выполнен расчет фич для: УникВидНоменкМенеджерГод
Выполнен расчет фич для: УникВидНоменкМенеджерВсе
Выполнен расчет фич для: Кол_воПартнеровУМенеджераНеделя
Выполнен расчет фич для: Кол_воПартнеровУМенеджераМесяц
Выполнен расчет фич для: Кол_воПартнеровУМенеджераКвартал
Выполнен расчет фич для: Кол_воПартнеровУМенеджераГод
Выполнен расчет фич для: Кол_воПартнеровУМенеджераВсего
Выполнен расчет фич для: МенеджерКол_воРегионовПоставкиНеделя
Выполнен расчет фич для: МенеджерКол_воРегионовПоставкиМесяц
Выполнен расчет фич для: МенеджерКол_воРегионовПоставкиКвартал
Выполнен расчет фич для: МенеджерКол_в

## Вид Номенклатуры - как основной признак

In [13]:
# Количество Регионов с поставляемой Номенклатурой
makePriznak(timeFile, "week", "VidNomenklatury", 'BiznesRegion', "nunique", 'ОхватРегионовВидНоменклНеделя')
makePriznak(timeFile, "months", "VidNomenklatury", 'BiznesRegion', "nunique", 'ОхватРегионовВидНоменклМесяц')
makePriznak(timeFile, "quarted", "VidNomenklatury", 'BiznesRegion', "nunique", 'ОхватРегионовВидНоменклКвартал')
makePriznak(timeFile, "year", "VidNomenklatury", 'BiznesRegion', "nunique", 'ОхватРегионовВидНоменклГод')
makePriznak(timeFile, "all", "VidNomenklatury", 'BiznesRegion', "nunique", 'ОхватРегионовВидНоменклВсего')

# Количество запросов на Номенклатуру (популярность Номенклатуры)
makePriznak(timeFile, "week", "VidNomenklatury", 'LinkOrder', "nunique", 'Кол_воЗаказовДляВидНоменклНеделя')
makePriznak(timeFile, "months", "VidNomenklatury", 'LinkOrder', "nunique", 'Кол_воЗаказовДляВидНоменклМесяц')
makePriznak(timeFile, "quarted", "VidNomenklatury", 'LinkOrder', "nunique", 'Кол_воЗаказовДляВидНоменклКвартал')
makePriznak(timeFile, "year", "VidNomenklatury", 'LinkOrder', "nunique", 'Кол_воЗаказовДляВидНоменклГод')
makePriznak(timeFile, "all", "VidNomenklatury", 'LinkOrder', "nunique", 'Кол_воЗаказовДляВидНоменклВсего')

# Количество продаж на Номенклатуру (популярность Номенлатуры)
makePriznak(timeFile, "week", "VidNomenklatury", 'RealizSsilka', "nunique", 'Кол_воРеализДляВидНоменклНеделя')
makePriznak(timeFile, "months", "VidNomenklatury", 'RealizSsilka', "nunique", 'Кол_воРеализДляВидНоменклМесяц')
makePriznak(timeFile, "quarted", "VidNomenklatury", 'RealizSsilka', "nunique", 'Кол_воРеализДляВидНоменклКвартал')
makePriznak(timeFile, "year", "VidNomenklatury", 'RealizSsilka', "nunique", 'Кол_воРеализДляВидНоменклГод')
makePriznak(timeFile, "all", "VidNomenklatury", 'RealizSsilka', "nunique", 'Кол_воРеализДляВидНоменклВсего')

Выполнен расчет фич для: ОхватРегионовВидНоменклНеделя
Выполнен расчет фич для: ОхватРегионовВидНоменклМесяц
Выполнен расчет фич для: ОхватРегионовВидНоменклКвартал
Выполнен расчет фич для: ОхватРегионовВидНоменклГод
Выполнен расчет фич для: ОхватРегионовВидНоменклВсего
Выполнен расчет фич для: Кол_воЗаказовДляВидНоменклНеделя
Выполнен расчет фич для: Кол_воЗаказовДляВидНоменклМесяц
Выполнен расчет фич для: Кол_воЗаказовДляВидНоменклКвартал
Выполнен расчет фич для: Кол_воЗаказовДляВидНоменклГод
Выполнен расчет фич для: Кол_воЗаказовДляВидНоменклВсего
Выполнен расчет фич для: Кол_воРеализДляВидНоменклНеделя
Выполнен расчет фич для: Кол_воРеализДляВидНоменклМесяц
Выполнен расчет фич для: Кол_воРеализДляВидНоменклКвартал
Выполнен расчет фич для: Кол_воРеализДляВидНоменклГод
Выполнен расчет фич для: Кол_воРеализДляВидНоменклВсего


## Номенклатура - как основной признак

In [14]:
# Количество Регионов с поставляемой Номенклатурой
makePriznak(timeFile, "week", "NomenklaturaKodRealAndZakaz", 'BiznesRegion', "nunique", 'ОхватРегионовНоменклатуройНеделя')
makePriznak(timeFile, "months", "NomenklaturaKodRealAndZakaz", 'BiznesRegion', "nunique", 'ОхватРегионовНоменклатуройМесяц')
makePriznak(timeFile, "quarted", "NomenklaturaKodRealAndZakaz", 'BiznesRegion', "nunique", 'ОхватРегионовНоменклатуройКвартал')
makePriznak(timeFile, "year", "NomenklaturaKodRealAndZakaz", 'BiznesRegion', "nunique", 'ОхватРегионовНоменклатуройГод')
makePriznak(timeFile, "all", "NomenklaturaKodRealAndZakaz", 'BiznesRegion', "nunique", 'ОхватРегионовНоменклатуройВсего')

# Количество запросов на Номенклатуру (популярность Номенклатуры)
makePriznak(timeFile, "week", "NomenklaturaKodRealAndZakaz", 'LinkOrder', "nunique", 'Кол_воЗаказовДляНоменклНеделя')
makePriznak(timeFile, "months", "NomenklaturaKodRealAndZakaz", 'LinkOrder', "nunique", 'Кол_воЗаказовДляНоменклМесяц')
makePriznak(timeFile, "quarted", "NomenklaturaKodRealAndZakaz", 'LinkOrder', "nunique", 'Кол_воЗаказовДляНоменклКвартал')
makePriznak(timeFile, "year", "NomenklaturaKodRealAndZakaz", 'LinkOrder', "nunique", 'Кол_воЗаказовДляНоменклГод')
makePriznak(timeFile, "all", "NomenklaturaKodRealAndZakaz", 'LinkOrder', "nunique", 'Кол_воЗаказовДляНоменклВсего')

# Количество продаж на Номенклатуру (популярность Номенлатуры)
makePriznak(timeFile, "week", "NomenklaturaKodRealAndZakaz", 'RealizSsilka', "nunique", 'Кол_воРеализДляНоменклНеделя')
makePriznak(timeFile, "months", "NomenklaturaKodRealAndZakaz", 'RealizSsilka', "nunique", 'Кол_воРеализДляНоменклМесяц')
makePriznak(timeFile, "quarted", "NomenklaturaKodRealAndZakaz", 'RealizSsilka', "nunique", 'Кол_воРеализДляНоменклКвартал')
makePriznak(timeFile, "year", "NomenklaturaKodRealAndZakaz", 'RealizSsilka', "nunique", 'Кол_воРеализДляНоменклГод')
makePriznak(timeFile, "all", "NomenklaturaKodRealAndZakaz", 'RealizSsilka', "nunique", 'Кол_воРеализДляНоменклВсего')

Выполнен расчет фич для: ОхватРегионовНоменклатуройНеделя
Выполнен расчет фич для: ОхватРегионовНоменклатуройМесяц
Выполнен расчет фич для: ОхватРегионовНоменклатуройКвартал
Выполнен расчет фич для: ОхватРегионовНоменклатуройГод
Выполнен расчет фич для: ОхватРегионовНоменклатуройВсего
Выполнен расчет фич для: Кол_воЗаказовДляНоменклНеделя
Выполнен расчет фич для: Кол_воЗаказовДляНоменклМесяц
Выполнен расчет фич для: Кол_воЗаказовДляНоменклКвартал
Выполнен расчет фич для: Кол_воЗаказовДляНоменклГод
Выполнен расчет фич для: Кол_воЗаказовДляНоменклВсего
Выполнен расчет фич для: Кол_воРеализДляНоменклНеделя
Выполнен расчет фич для: Кол_воРеализДляНоменклМесяц
Выполнен расчет фич для: Кол_воРеализДляНоменклКвартал
Выполнен расчет фич для: Кол_воРеализДляНоменклГод
Выполнен расчет фич для: Кол_воРеализДляНоменклВсего


## КредитИлиПредоплата - как основной признак

In [15]:
# Количество Заказов по виду оплаты
makePriznak(timeFile, "week", "KreditIliPredoplata", 'LinkOrder', "nunique", 'Кол_воЗаказВидОплатыНеделя')
makePriznak(timeFile, "months", "KreditIliPredoplata", 'LinkOrder', "nunique", 'Кол_воЗаказВидОплатыМесяц')
makePriznak(timeFile, "quarted", "KreditIliPredoplata", 'LinkOrder', "nunique", 'Кол_воЗаказВидОплатыКвартал')
makePriznak(timeFile, "year", "KreditIliPredoplata", 'LinkOrder', "nunique", 'Кол_воЗаказВидОплатыГод')
makePriznak(timeFile, "all", "KreditIliPredoplata", 'LinkOrder', "nunique", 'Кол_воЗаказВидОплатыВсего')
                      
# Количество Реализ по виду оплаты
makePriznak(timeFile, "week", "KreditIliPredoplata", 'RealizSsilka', "nunique", 'Кол_воРеализВидОплатыНеделя')
makePriznak(timeFile, "months", "KreditIliPredoplata", 'RealizSsilka', "nunique", 'Кол_воРеализВидОплатыМесяц')
makePriznak(timeFile, "quarted", "KreditIliPredoplata", 'RealizSsilka', "nunique", 'Кол_воРеализВидОплатыКвартал')
makePriznak(timeFile, "year", "KreditIliPredoplata", 'RealizSsilka', "nunique", 'Кол_воРеализВидОплатыГод')
makePriznak(timeFile, "all", "KreditIliPredoplata", 'RealizSsilka', "nunique", 'Кол_воРеализВидОплатыВсего')

# Количество ВидовНоменлатуры по виду оплаты
makePriznak(timeFile, "week", "KreditIliPredoplata", 'VidNomenklatury', "nunique", 'ВидовНоменклатВидОплатыНеделя')
makePriznak(timeFile, "months", "KreditIliPredoplata", 'VidNomenklatury', "nunique", 'ВидовНоменклатВидОплатыМесяц')
makePriznak(timeFile, "quarted", "KreditIliPredoplata", 'VidNomenklatury', "nunique", 'ВидовНоменклатВидОплатыКвартал')
makePriznak(timeFile, "year", "KreditIliPredoplata", 'VidNomenklatury', "nunique", 'ВидовНоменклатВидОплатыГод')
makePriznak(timeFile, "all", "KreditIliPredoplata", 'VidNomenklatury', "nunique", 'ВидовНоменклатВидОплатыВсего')

# Количество Номенлатуры по виду оплаты
makePriznak(timeFile, "week", "KreditIliPredoplata", 'NomenklaturaKodRealAndZakaz', "nunique", 'НоменклатВидОплатыНеделя')
makePriznak(timeFile, "months", "KreditIliPredoplata", 'NomenklaturaKodRealAndZakaz', "nunique", 'НоменклатВидОплатыМесяц')
makePriznak(timeFile, "quarted", "KreditIliPredoplata", 'NomenklaturaKodRealAndZakaz', "nunique", 'НоменклатВидОплатыКвартал')
makePriznak(timeFile, "year", "KreditIliPredoplata", 'NomenklaturaKodRealAndZakaz', "nunique", 'НоменклатВидОплатыГод')
makePriznak(timeFile, "all", "KreditIliPredoplata", 'NomenklaturaKodRealAndZakaz', "nunique", 'НоменклатВидОплатыВсего')

# Количество БизнесРегионов по виду оплаты
makePriznak(timeFile, "week", "KreditIliPredoplata", 'BiznesRegion', "nunique", 'БизнесРегионВидОплатыНеделя')
makePriznak(timeFile, "months", "KreditIliPredoplata", 'BiznesRegion', "nunique", 'БизнесРегионВидОплатыМесяц')
makePriznak(timeFile, "quarted", "KreditIliPredoplata", 'BiznesRegion', "nunique", 'БизнесРегионВидОплатыКвартал')
makePriznak(timeFile, "year", "KreditIliPredoplata", 'BiznesRegion', "nunique", 'БизнесРегионВидОплатыГод')
makePriznak(timeFile, "all", "KreditIliPredoplata", 'BiznesRegion', "nunique", 'БизнесРегионВидОплатыВсего')

Выполнен расчет фич для: Кол_воЗаказВидОплатыНеделя
Выполнен расчет фич для: Кол_воЗаказВидОплатыМесяц
Выполнен расчет фич для: Кол_воЗаказВидОплатыКвартал
Выполнен расчет фич для: Кол_воЗаказВидОплатыГод
Выполнен расчет фич для: Кол_воЗаказВидОплатыВсего
Выполнен расчет фич для: Кол_воРеализВидОплатыНеделя
Выполнен расчет фич для: Кол_воРеализВидОплатыМесяц
Выполнен расчет фич для: Кол_воРеализВидОплатыКвартал
Выполнен расчет фич для: Кол_воРеализВидОплатыГод
Выполнен расчет фич для: Кол_воРеализВидОплатыВсего
Выполнен расчет фич для: ВидовНоменклатВидОплатыНеделя
Выполнен расчет фич для: ВидовНоменклатВидОплатыМесяц
Выполнен расчет фич для: ВидовНоменклатВидОплатыКвартал
Выполнен расчет фич для: ВидовНоменклатВидОплатыГод
Выполнен расчет фич для: ВидовНоменклатВидОплатыВсего
Выполнен расчет фич для: НоменклатВидОплатыНеделя
Выполнен расчет фич для: НоменклатВидОплатыМесяц
Выполнен расчет фич для: НоменклатВидОплатыКвартал
Выполнен расчет фич для: НоменклатВидОплатыГод
Выполнен расчет

# Сохраним Файл с фичами

In [16]:
with pandas.ExcelWriter(f'Продажи с фичами {datetime.today().strftime("%Y-%m-%d")}.xlsx') as writer:
    timeFile.reset_index(drop=True).to_excel(writer, sheet_name='Данные для анализа')

In [17]:
timeFile.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146016 entries, 0 to 146015
Columns: 214 entries, AktVipRabot to БизнесРегионВидОплатыВсего
dtypes: datetime64[ns](5), float64(11), int64(5), object(193)
memory usage: 243.5+ MB
